In [14]:
import pandas as pd
import plotly.offline as offline 
import plotly.graph_objs as go
from plotly import subplots

# 일자(n,m,t)에 따른 Stochastic(KDJ)의 값을 구하기 위해 함수형태로 만듬 
def get_stochastic(df, n=15, m=5, t=3):
    
    # 입력받은 값이 dataframe이라는 것을 정의해줌
    df = pd.DataFrame(df)
    
    # n일중 최고가
    ndays_high = df.high.rolling(window=n, min_periods=1).max()
    # n일중 최저가
    ndays_low = df.low.rolling(window=n, min_periods=1).min()
 
    # Fast%K 계산
    kdj_k = ((df.close - ndays_low) / (ndays_high - ndays_low))*100
    # Fast%D (=Slow%K) 계산
    kdj_d = kdj_k.ewm(span=m).mean()
    # Slow%D 계산
    kdj_j = kdj_d.ewm(span=t).mean()
 
    # dataframe에 컬럼 추가
    df = df.assign(kdj_k=kdj_k, kdj_d=kdj_d, kdj_j=kdj_j).dropna()
    
    return df

In [48]:

extract_list = ['CSA 코스믹', 'DSR', 'LS산전', 'SK네트웍스', '이리츠코크렙', '인터지스']
 
code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]
 
# 종목코드가 6자리이기 때문에 6자리를 맞춰주기 위해 설정해줌
code_df.종목코드 = code_df.종목코드.map('{:06d}'.format)
 
# 우리가 필요한 것은 회사명과 종목코드이기 때문에 필요없는 column들은 제외해준다.
code_df = code_df[['회사명', '종목코드']]
 
# 한글로된 컬럼명을 영어로 바꿔준다.
code_df = code_df.rename(columns={'회사명': 'name', '종목코드': 'code'})
# code_df.head()

# 종목 이름을 입력하면 종목에 해당하는 코드를 불러와
# 네이버 금융(http://finance.naver.com)에 넣어줌
def get_url(item_name, code_df):

    code = code_df.query("name=='{}'".format(item_name))['code'].to_string(index=False).strip()
    url = 'http://finance.naver.com/item/sise_day.nhn?code={code}'.format(code=code)
    
    print("요청 URL = {}".format(url))
    return url

for i in code_df.index:
# for i in range(10):
    if code_df.iloc[i]['name'].strip() in extract_list:
        continue
        
    code = code_df.iloc[i]['code'].strip()
    url = 'http://finance.naver.com/item/sise_day.nhn?code={code}'.format(code=code)
    print("요청 URL = {}".format(url), code_df.iloc[i]['name'].strip())
    df = pd.DataFrame()
    
    for page in range(1, 11):
        pg_url = '{url}&page={page}'.format(url=url, page=page)
        df = df.append(pd.read_html(pg_url, header=0)[0], ignore_index=True)
    
    # df.dropna()를 이용해 결측값 있는 행 제거
    df = df.dropna()
    
    # 한글로 된 컬럼명을 영어로 바꿔줌
    df = df.rename(columns= {'날짜': 'date', '종가': 'close', '전일비': 'diff', 
                         '시가': 'open', '고가': 'high', '저가': 'low', '거래량': 'volume'})
    
    # 데이터의 타입을 int형으로 바꿔줌
    df[['close', 'diff', 'open', 'high', 'low', 'volume']] = \
    df[['close', 'diff', 'open', 'high', 'low', 'volume']].astype(int)
 
    # 컬럼명 'date'의 타입을 date로 바꿔줌
    df['date'] = pd.to_datetime(df['date'])
 
    # 일자(date)를 기준으로 오름차순 정렬
    df = df.sort_values(by=['date'], ascending=True)
    df = get_stochastic(df)
    try:
        if (int(float((df.loc[1:1, ['kdj_d']]['kdj_d'].to_string().split('1 ')[1].strip()))) < 20):
            print( '-', df.loc[1:1, ['kdj_d']])
        if (int(float((df.loc[1:1, ['kdj_d']]['kdj_d'].to_string().split('1 ')[1].strip()))) > 80):
            print( '+', df.loc[1:1, ['kdj_d']])
    except ValueError as e:
        pass

    

요청 URL = http://finance.naver.com/item/sise_day.nhn?code=001250 GS글로벌
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=082740 HSD엔진
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=011070 LG이노텍
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=010120 LS산전
-       kdj_d
1  4.608124
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=010060 OCI
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=100840 S&TC
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=064960 S&T모티브
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=001740 SK네트웍스
-       kdj_d
1  4.418102
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=068400 SK렌터카
-        kdj_d
1  17.649024
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=096770 SK이노베이션
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=011810 STX
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=024070 WISCOM
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=011420 갤럭시아에스엠
-        

-       kdj_d
1  8.363285
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=057050 현대홈쇼핑
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=111110 호전실업
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=008770 호텔신라
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=298040 효성중공업
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=051500 CJ프레시웨이
-       kdj_d
1  9.739603
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=245620 EDGC
-        kdj_d
1  18.441924
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=130500 GH신소재
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=083450 GST
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=067290 JW신약
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=024120 KB오토시스
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=337450 SK5호스팩
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=089230 THE E&M
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=032540 TJ미디어
요청 URL = h

IndexError: list index out of range